## Animated Speed Line Chart with All the Data Points

In [15]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import sys
import os

sys.path.append(os.path.abspath("..")) 
FILE_PATH = '../data/bird_migration.csv' 


### **2. Data Loading & Preprocessing**
In this step, we load the raw bird migration data and prepare it for animation.

* **Load Data:** Reads the CSV file from the specified path.
* **Time Formatting:** Converts the `date_time` column to proper datetime objects for sorting.
* **Cleaning:** Fills missing speed values with 0 and sorts the dataset chronologically.
* **Global Variables:** Calculates the **Start** and **End** dates of the entire dataset to synchronize the animation timeline.

In [16]:
try:
    df = pd.read_csv(FILE_PATH)
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['speed_2d'] = df['speed_2d'].fillna(0)
    df = df.sort_values('date_time')
    
    # We just create a copy to keep variable names consistent for the app logic
    df_all_data = df.copy()
    
    # Global Time Range
    GLOBAL_MIN_DATE = df_all_data['date_time'].min()
    GLOBAL_MAX_DATE = df_all_data['date_time'].max()
    ALL_BIRDS = sorted(df_all_data['bird_name'].unique().tolist())
    
    print(f"Data Loaded: {len(df_all_data)} points. Range: {GLOBAL_MIN_DATE} to {GLOBAL_MAX_DATE}")

except Exception as e:
    print(f"Critical Error: {e}")
    df_all_data = pd.DataFrame()
    ALL_BIRDS = []
    GLOBAL_MIN_DATE = pd.Timestamp.now()
    GLOBAL_MAX_DATE = pd.Timestamp.now()


Data Loaded: 61920 points. Range: 2013-08-15 00:01:08+00:00 to 2014-04-30 23:59:34+00:00


### **3. Animation Setup (Slider Marks)**
Here we configure the timeline slider. Since the animation is driven by a percentage (0-100%), we need to calculate and display the actual **Dates** that correspond to those positions.

* **Total Duration:** Calculates the total time span of the dataset in seconds.
* **Mark Generation:** Creates 6 evenly spaced labels (0%, 20%, 40%, etc.) by adding a fraction of the total duration to the start date.
* **Formatting:** Formats the labels as "Month Day" (e.g., *Aug 15*) with the Year on a new line for better readability.


In [ ]:
total_seconds = (GLOBAL_MAX_DATE - GLOBAL_MIN_DATE).total_seconds()

if total_seconds > 0:
    # Create marks at 0%, 20%, 40%, 60%, 80%, 100%
    for i in range(0, 101, 20): 
        fraction = i / 100
        # Calculate exact date at this percentage
        date_mark = GLOBAL_MIN_DATE + pd.Timedelta(seconds=total_seconds * fraction)
        
        # Style the label (e.g. "Aug 15" on top, "2013" below)
        slider_marks[i] = {
            'label': date_mark.strftime('%b %d\n%Y'), 
            'style': {'fontSize': '12px', 'whiteSpace': 'pre-line'}
        }
else:
    # Fallback if no data
    slider_marks = {0: 'Start', 100: 'End'}



### **4. App Layout**
This section defines the visual structure of the dashboard using Dash Bootstrap Components.

* **Header:** A simple centered title for the application.
* **Chart Card:** A container for the `dcc.Graph`. We set `animate=False` here to ensure smooth custom updates via the slider, avoiding the default Plotly animation engine which can be buggy with large datasets.
* **Controls Card:** Contains the interactive elements:
    * **Dropdown:** Allows filtering by specific bird names (defaults to empty).
    * **Play Button:** Toggles the animation loop.
    * **Slider:** Represents the timeline (0-100%). It includes the custom date marks generated in the previous step.
    * **Interval:** A hidden timer component (`dcc.Interval`) that triggers every 100ms to auto-advance the slider when "Play" is active.


In [18]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUMEN])

app.layout = dbc.Container([
    
    html.H2("Speed Animation", className="my-4 text-center"),
    
    dbc.Card([
        dbc.CardBody([
            # animate=False is safer for heavy datasets
            dcc.Graph(id='speed-line-chart', animate=False) 
        ])
    ], className="shadow-sm mb-4"),
    
    dbc.Card([
        dbc.CardBody([
            html.Label("Select Birds:", className="fw-bold"),
            dcc.Dropdown(
                id='bird-name-filter',
                options=[{'label': b, 'value': b} for b in ALL_BIRDS],
                value=[], 
                multi=True
            ),
            html.Hr(),
            
            # Controls Row
            dbc.Row([
                dbc.Col(
                    html.Button('Play', id='play-button', n_clicks=0, className="btn btn-success w-100"),
                    width=2
                ),
                dbc.Col(
                    dcc.Slider(
                        id='animation-slider',
                        min=0, max=100, step=0.5, 
                        value=0, 
                        marks=slider_marks,
                        updatemode='drag' 
                    ),
                    width=10
                )
            ], align="center"),
            
            dcc.Interval(id='auto-stepper', interval=100, n_intervals=0, disabled=True)
        ])
    ], className="bg-light mb-5")
], fluid=True)



### **5. Application Logic (Callbacks)**
These functions manage the interactivity of the dashboard.

1.  **Toggle Play/Pause:** Listens to the `Play` button click. It toggles the `disabled` state of the `dcc.Interval` component to start or stop the automatic updates.
2.  **Advance Timer:** This is the animation engine. Triggered by the interval timer every 100ms, it increments the slider value by **0.5%** to create a smooth progression. It loops back to 0 when it reaches 100%.
3.  **Update Chart:** The core visualization logic.
    * **Inputs:** Reacts to changes in the **Slider** (Time) and the **Dropdown** (Selected Birds).
    * **Data Slicing:** Calculates the specific *Cutoff Date* corresponding to the slider's position (e.g., 50% = Middle Date). It then filters the dataframe to include only flight paths recorded *before* that date.
    * **Dynamic Plotting:** Loops through each selected bird and draws a separate line trace.
    * **Locked Axes:** Crucially, the X-axis (Time) and Y-axis (Speed) ranges are fixed to the global minimums and maximums. This prevents the chart from "jumping" or rescaling wildly as new data points appear during the animation.


In [19]:

@app.callback(
    [Output('auto-stepper', 'disabled'), Output('play-button', 'children')],
    [Input('play-button', 'n_clicks')], [State('auto-stepper', 'disabled')]
)
def toggle_play(n_clicks, current_disabled):
    if n_clicks:
        return not current_disabled, "Pause" if current_disabled else "Play"
    return True, "Play"

@app.callback(
    Output('animation-slider', 'value'),
    [Input('auto-stepper', 'n_intervals')], [State('animation-slider', 'value')]
)
def advance_timer(n, current_val):
    if current_val >= 100: return 0 
    return current_val + 0.5 

@app.callback(
    Output('speed-line-chart', 'figure'),
    [Input('animation-slider', 'value'), Input('bird-name-filter', 'value')]
)
def update_chart(slider_value, selected_birds):
    # Handle Empty State
    if df_all_data.empty or not selected_birds:
        return go.Figure().update_layout(
            title="Select birds to display data", 
            xaxis={'visible': True}, yaxis={'visible': True},
            template="plotly_white"
        )
    
    if isinstance(selected_birds, str): selected_birds = [selected_birds]

    # Calculate Cutoff Time (This stays exactly the same!)
    fraction = slider_value / 100
    cutoff_time = GLOBAL_MIN_DATE + pd.Timedelta(seconds=total_seconds * fraction)
    
    # Setup Figure
    fig = go.Figure()
    bird_colors = {'Eric': '#FF5733', 'Nico': '#33FF57', 'Sanne': '#3357FF'}
    
    # Calculate Max Y based on selected birds (for stable axis)
    visible_data = df_all_data[df_all_data['bird_name'].isin(selected_birds)]
    max_y = visible_data['speed_2d'].max() if not visible_data.empty else 100

    # Draw Lines
    for bird in selected_birds:
        bird_df = df_all_data[df_all_data['bird_name'] == bird]
        
        # Slice Data: Just filter by date, no extra grouping!
        df_slice = bird_df[bird_df['date_time'] <= cutoff_time]
        
        if not df_slice.empty:
            fig.add_trace(go.Scatter(
                x=df_slice['date_time'],
                y=df_slice['speed_2d'],
                mode='lines',
                name=bird,
                line=dict(color=bird_colors.get(bird, '#999'), width=1.5), # Thinner line for high-res data
                opacity=0.8
            ))

    # Final Layout (Locked Axes)
    fig.update_layout(
        title=f"<b>Flight Speed ({cutoff_time.strftime('%Y-%m-%d')})</b>",
        xaxis=dict(
            title="Date",
            range=[GLOBAL_MIN_DATE, GLOBAL_MAX_DATE], 
            showgrid=False
        ),
        yaxis=dict(
            title="Speed (km/h)", 
            range=[0, max_y * 1.1], 
            showgrid=True
        ),
        template="plotly_white",
        height=400,
        margin=dict(l=40, r=40, t=60, b=40),
        legend=dict(orientation="h",      
            y=1.02,             
            xanchor="right",
            x=1  ),
        hovermode="x unified",
        showlegend=True               
        )

    
    
    return fig


#### 5. Run Server


In [20]:
if __name__ == '__main__':
    app.run(debug=True, port=8053)